<a href="https://colab.research.google.com/github/bogus1aw/text-classification-benchmark/blob/main/M_herBERT_wiki_dataset_fraction_test_RAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HerBERT benchmark for wiki dataset 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# check available GPU
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla T4, 418.67, 15079 MiB


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime
import time

In [ ]:
%%capture
!pip install datasets transformers

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.is_available()

True

In [ ]:
def load_corpora_to_dataframe(corpora):
  data = open(corpora).read()
  labels, texts = [], []
  for i, line in enumerate(data.split("\n")):
      content = line.split()
      labels.append(content[0])
      texts.append(" ".join(content[1:]))
  return texts, labels
  # # create a dataframe using texts and labels
  # trainDF = pd.DataFrame()
  # trainDF['text'] = texts
  # trainDF['label'] = labels
  # return trainDF

timestamp = datetime.datetime.now().replace(microsecond=0).isoformat().replace(':', '-')
metrice_path = '/content/drive/MyDrive/metrics/herBERT_wiki_metrice_raw' + timestamp + '.txt'
#raw    
corpora = '/content/drive/MyDrive/master_datasets/wiki_preprocessed/wikiInOneFileDataset.txt'
#lemmas
# corpora = '/content/drive/MyDrive/master_datasets/wiki_preprocessed/wikiInOneFileDataset_lemmas.txt'

texts, labels = load_corpora_to_dataframe(corpora)

no_samples_per_class = [1, 3, 5, 8, 10, 20, 30, 60, 100, 200] ###### FINALL SAMPLES LIST  
no_samples_per_class = [60, 100, 200] ###### FINALL SAMPLES LIST  
# no_samples_per_class.reverse() 
repetitions = 1

# 1. load corpora
# 2. create training, test fractions
# 3. create specific No. per class fractions
# 4. create train, val sets
# 5. create new fresh model (tokenizer can stay the same)
# 6. tokenize and encode train, test 

In [ ]:
data_df = pd.DataFrame()
data_df['labels'] = labels
data_df['texts'] = texts

train, test = train_test_split(data_df, test_size=.2, random_state=42)

In [ ]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification, EvalPrediction


tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-base-cased", num_labels=34)
# tokenizer = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
# model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=24)

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing RobertaForSequenceClassification: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weigh

In [ ]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

def build_databases(train_data, test_data):
  # create train and validation dataset
  train_texts, val_texts, train_labels, val_labels = train_test_split(train_data['texts'].to_list(), train_data['labels'].to_list(), test_size=.2)

  max_length = 200
  train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
  val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)
  test_encodings = tokenizer(test_data['texts'].to_list(), truncation=True, padding=True, max_length=max_length)

  # encode labels
  train_labels = encoder.fit_transform(train_labels)
  val_labels = encoder.fit_transform(val_labels)
  test_labels = encoder.fit_transform(test_data['labels'].to_list())

  # build pyTorch dataset
  import torch

  class wikiDataset(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
          item['labels'] = torch.tensor(self.labels[idx])
          return item

      def __len__(self):
          return len(self.labels)

  train_dataset = wikiDataset(train_encodings, train_labels)
  val_dataset = wikiDataset(val_encodings, val_labels)
  test_dataset = wikiDataset(test_encodings, test_labels)
  return train_dataset, val_dataset, test_dataset

# train_dataset, val_dataset, test_dataset = build_databases()


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy = "epoch",
    num_train_epochs=10,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=20,                             
    logging_dir='./logs',            
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    adam_epsilon=1e-6,
    adam_beta1=0.9,
    adam_beta2=0.98,
    learning_rate=1e-5
)

from datasets import load_metric
import numpy as np
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    # print(predictions[:10])
    # print(labels[:10])
    return metric.compute(predictions=predictions, references=labels)

def get_trainer(model, train_dataset, val_dataset):
  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,             # evaluation dataset
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  return trainer


In [ ]:

for n_sample in no_samples_per_class:
  for repeat in range(repetitions):
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('training for n_sample=', n_sample)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-base-cased", num_labels=34) ######## Change num of labels !!!!!
    # model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=34) 
    dataset_fraction = train.groupby(['labels']).sample(n=n_sample, replace=True)
    train_dataset, val_dataset, test_dataset = build_databases(dataset_fraction, test)
    trainer = get_trainer(model=model, train_dataset=train_dataset, val_dataset=val_dataset)
    
    train_time_start = time.time()
    trainer.train()
    train_elapsed_time = time.time() - train_time_start
    
    # trainer.evaluate()
    accuracy = trainer.predict(test_dataset)
    to_save = f'n_samples_per_class={n_sample}, repeat={repeat}, time_elapsed={train_elapsed_time}, {accuracy.metrics}\n' 
    print(to_save)
    with open(metrice_path, 'a') as f:
      f.write(to_save)


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training for n_sample= 60
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing RobertaForSequenceClassification: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weigh

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,3.546500,3.570857,0.019608,5.812300,70.195000
2,3.575300,3.539426,0.019608,5.580300,73.115000
3,3.546600,3.538575,0.031863,5.504900,74.115000
4,3.529500,3.529719,0.066176,5.510100,74.046000
5,3.521600,3.517790,0.046569,5.470000,74.588000
6,3.480800,3.441092,0.115196,5.466300,74.639000
7,3.294200,3.366641,0.078431,5.473900,74.536000
8,3.241300,3.228203,0.093137,5.498200,74.206000
9,3.185500,3.159912,0.139706,5.468700,74.607000
10,3.117500,3.150743,0.147059,5.507100,74.086000


n_samples_per_class=60, repeat=0, time_elapsed=842.0071549415588, {'eval_loss': 3.1753089427948, 'eval_accuracy': 0.16122004357298475, 'eval_runtime': 18.5883, 'eval_samples_per_second': 74.079}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training for n_sample= 100
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing RobertaForSequenceClassification: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weigh

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,3.571700,3.544595,0.030882,9.255300,73.472000
2,3.551600,3.547118,0.023529,9.312700,73.019000
3,3.532400,3.526579,0.025000,9.319300,72.967000
4,3.406700,3.336347,0.094118,9.338600,72.816000
5,3.178500,3.151061,0.119118,9.328800,72.892000
6,2.959500,3.015701,0.175000,9.311100,73.031000
7,2.943700,2.862628,0.204412,9.306200,73.070000
8,2.757700,2.762715,0.220588,9.303700,73.090000
9,2.693600,2.701386,0.244118,9.282200,73.259000
10,2.642400,2.678516,0.241176,9.287300,73.218000


n_samples_per_class=100, repeat=0, time_elapsed=1354.6516554355621, {'eval_loss': 2.827895164489746, 'eval_accuracy': 0.2084241103848947, 'eval_runtime': 18.6004, 'eval_samples_per_second': 74.031}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training for n_sample= 200
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing RobertaForSequenceClassification: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weigh

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,3.541200,3.540754,0.033824,18.423500,73.819000
2,3.339600,3.356598,0.069853,18.507200,73.485000
3,2.902000,2.932433,0.158088,18.581800,73.190000
4,2.504200,2.572555,0.280882,18.562000,73.268000
5,2.268000,2.333473,0.297794,18.487400,73.564000
6,1.860400,1.929093,0.469853,18.516700,73.447000
7,1.619800,1.779403,0.522059,18.554300,73.299000
8,1.542900,1.620142,0.616912,18.585300,73.176000
9,1.383200,1.510672,0.633824,18.518900,73.438000
10,1.308100,1.458980,0.656618,18.538500,73.361000


n_samples_per_class=200, repeat=0, time_elapsed=2626.8549106121063, {'eval_loss': 1.7918285131454468, 'eval_accuracy': 0.5090777051561365, 'eval_runtime': 18.6332, 'eval_samples_per_second': 73.9}

